# Step 1. assess the reference human corpus

to recreate size distribution

In [ ]:
import pandas as pd

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
!pip install openai

In [ ]:
import openai

In [ ]:
from tqdm import tqdm

In [ ]:
import time

In [ ]:
import os

In [ ]:
import random

In [ ]:
#!git clone https://github.com/lucagiovannini7/ai-storyteller #this one's private
!git clone https://github.com/DanilSko/human_part_of_aist

In [ ]:
folder = 'human_part_of_aist/bm'

In [ ]:
def measure_file_sizes(folder):
    files = os.listdir(folder)
    sizes = []
    for filename in files:
        filepath = os.path.join(folder, filename)
        with open(filepath) as openfile:
            text = openfile.read()
        lenght = len(text.split())
        sizes.append(lenght)
    df = pd.DataFrame(sizes, files)
    df.columns = ['len_words']
    return df

In [ ]:
df = measure_file_sizes(folder)

In [ ]:
df

In [ ]:
df.plot.hist(bins=30);

In [ ]:
df['generated'] = False

In [ ]:
df

### Get lengths pro author

In [ ]:
authors = pd.Series(df.index.str.split('_')).apply(lambda x: x[0])

In [ ]:
df['author'] = list(authors)

In [ ]:
lengths = {'Edgar Poe': df[df['author'] == 'Poe']['len_words'],
           'Arnold Bennett': df[df['author'] == 'Bennett']['len_words'],
           'Katherine Mansfield': df[df['author'] == 'Mansfield']['len_words']}

# Step 2. Generation

In [ ]:
openai.api_key= #INSERT YOUR KEY

### Functions

In [ ]:
def prepare_promt(writer):
    starter_prompt = f"""Assume you are writer {writer}. Write the first part of a short story in prose in your own authorial voice"""
    messages=[
            {"role": "system", "content": f"You are writer {writer}"},
            {"role": "user", "content": starter_prompt},
            ]
    return messages

In [ ]:
def prepare_next_promt(writer):
    next_prompt = f"""Write the next part of the same short story. Keep assuming you are writer {writer}."""
    return next_prompt

In [ ]:
def ask_gpt(messages):
    try:
        response = openai.ChatCompletion.create(
        model="gpt-4", # "gpt-3.5-turbo"
        messages=messages
        )
    except:
        time.sleep(15)
        response = openai.ChatCompletion.create(
        model="gpt-4", # "gpt-3.5-turbo"
        messages=messages
        )
    return response

### Generation

In [ ]:
!mkdir generated

In [ ]:
authors = ['Arnold Bennett', 'Edgar Poe', 'Katherine Mansfield']

In [ ]:
def get_texts_from_author(author, author_lens, iters=10, sleeptime = 15):
    author_surname_lower = author.split()[-1].lower()
    for i in tqdm(range(iters)):
        filename = f'{author_surname_lower}_{i}.txt'
        print(filename)
        this_len = author_lens[i]
        result_text = generate_single_text(author, this_len = this_len, sleeptime = sleeptime)
        with open(f'generated/{filename}', 'w') as output:
            output.write(result_text)
            output.write('\n')

In [ ]:
def generate_single_text(author, this_len, sleeptime):
    print(f'the generated text size shall be close to the length of {this_len} words')
    time.sleep(sleeptime)
    messages = prepare_promt(author)
    response = ask_gpt(messages)
    response_text = response['choices'][0]['message']['content']
    text = response_text
    while len(text.split()) < (this_len + random.randint(-100,100)):
        continuation_prompt = prepare_next_promt(author)
        messages.append({"role": "assistant", "content": response_text},)
        messages.append({"role": "user", "content": continuation_prompt},)
        if len(messages) > 4:
            #print(messages)
            messages.pop(1)
            messages.pop(1)
            #print(messages)
            #print('Removed old dialogue parts')
            #print(len(messages))
        response = ask_gpt(messages)
        response_text = response['choices'][0]['message']['content']
        text += response_text
        time.sleep(sleeptime)
    print(f'resulting text length: {len(text.split())}')
    return text

In [ ]:
for author in authors:
    author_lens = lengths[author]
    get_texts_from_author(author, author_lens)

In [ ]:
author = 'Edgar Poe'
author_lens = lengths['Edgar Poe']
sleeptime = 5
author_surname_lower = author.split()[-1].lower()
for i in tqdm(range(8,10)):
    filename = f'{author_surname_lower}_{i}.txt'
    print(filename)
    this_len = author_lens[i]
    result_text = generate_single_text(author, this_len = this_len, sleeptime = sleeptime)
    with open(f'generated/{filename}', 'w') as output:
        output.write(result_text)
        output.write('\n')

Add generated stuff

In [ ]:
folder = 'generated'

In [ ]:
dfgen = measure_file_sizes(folder)

In [ ]:
len(dfgen)

In [ ]:
len(df)

In [ ]:
plt.hist(df['len_words'], bins=30)
plt.hist(dfgen['len_words'], bins=30, alpha=0.5)
plt.legend(labels = ['human', 'gpt'])
plt.xlabel('Length in words')
plt.ylabel('Number of texts')
plt.show;

In [ ]:
dfall = pd.concat((df, dfgen))

In [ ]:
dfall.groupby('generated')